In [1]:
!pip install pyLDAvis

In [13]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
import time
import gensim
import pyLDAvis.gensim
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#read the csv file with amazon reviews
reviews_df=pd.read_csv('/content/reviews.csv',error_bad_lines=False)
print(reviews_df.head(6))
print("----------")
print(reviews_df.dtypes)
print("----------")
print(reviews_df.shape)


                                             Reviews
0  I thought it would be as big as small paper bu...
1  This kindle is light and easy to use especiall...
2  Didnt know how much i'd use a kindle so went f...
3  I am 100 happy with my purchase. I caught it o...
4  Solid entry level Kindle. Great for kids. Gift...
5  This make an excellent ebook reader. Don't exp...
----------
Reviews    object
dtype: object
----------
(5000, 1)


In [5]:
def initial_clean(text):
    """
    Function to clean text-remove punctuations, lowercase text etc.
    """
    text = re.sub("[^a-zA-Z ]", "", text)
    text = text.lower()  # lower case text
    text = nltk.word_tokenize(text)
    return (text)

In [6]:

stop_words = stopwords.words('english')
# adding some more stop words that doesn't convey much meaning in terms of reviews feel free to extend or reduce this list
stop_words.extend(['news', 'say','use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 
                   'good', 'go', 'get', 'do','took','time','year',
                 'done', 'try', 'many', 'some','nice', 'thank', 'think', 'see', 'rather',
                  'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 
                   'even', 'right', 'line','even', 'also', 'may', 'take', 'come',
                   'new','said', 'like','people'])

In [7]:
def remove_stop_words(text):
     return [word for word in text if word not in stop_words]

In [8]:
stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # no single letter words
    except IndexError:
        pass

    return text

In [9]:
def apply_all(text):
    """
    This function applies all the functions above into one
    """
    return stem_words(remove_stop_words(initial_clean(text)))

In [10]:

t1 = time.time()
reviews_df['tokenized_reviews'] = reviews_df['Reviews'].apply(apply_all)
t2 = time.time()
print("Time to clean and tokenize", len(reviews_df), "reviews:", (t2-t1)/60, "min") #Time to clean and tokenize 3209 reviews: 0.21254388093948365 min

print('\n')
print("reviews with their respective tokenize version:" )
print(reviews_df.head(5))

Time to clean and tokenize 5000 reviews: 0.04674838383992513 min


reviews with their respective tokenize version:
                                             Reviews                                  tokenized_reviews
0  I thought it would be as big as small paper bu...  [thought, big, small, paper, turn, palm, small...
1  This kindle is light and easy to use especiall...                      [kindl, light, especi, beach]
2  Didnt know how much i'd use a kindle so went f...  [didnt, much, id, kindl, went, lower, end, im,...
3  I am 100 happy with my purchase. I caught it o...  [happi, purchas, caught, sale, realli, price, ...
4  Solid entry level Kindle. Great for kids. Gift...  [solid, entri, level, kindl, great, kid, gift,...


In [11]:
#LDA
#Create a Gensim dictionary from the tokenized data
tokenized = reviews_df['tokenized_reviews']
#Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(tokenized)
#Filter terms which occurs in less than 1 review and more than 80% of the reviews.
dictionary.filter_extremes(no_below=1, no_above=0.8)
#convert the dictionary to a bag of words corpus
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized]
print(corpus[:1])

print([[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1)]]
[[('big', 1), ('comfort', 1), ('definit', 1), ('instead', 1), ('kindl', 1), ('palm', 1), ('paper', 1), ('paperwhit', 1), ('read', 1), ('recommend', 1), ('regular', 1), ('small', 2), ('thought', 1), ('turn', 1)]]


In [29]:
#LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 7, id2word=dictionary, passes=15, alpha='auto', eta=0.61)
#saving the model
ldamodel.save('model_combined.gensim')
topics = ldamodel.print_topics(num_words=4)
print('\n')
print("Now printing the topics and their composition")
print("This output shows the Topic-Words matrix for the 7 topics created and the 4 words within each topic")
for topic in topics:
   print(topic)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt



Now printing the topics and their composition
This output shows the Topic-Words matrix for the 7 topics created and the 4 words within each topic
(0, '0.024*"devic" + 0.021*"home" + 0.010*"app" + 0.010*"alexa"')
(1, '0.032*"great" + 0.032*"tablet" + 0.016*"amazon" + 0.014*"price"')
(2, '0.054*"kindl" + 0.043*"read" + 0.035*"book" + 0.016*"fire"')
(3, '0.008*"gener" + 0.007*"th" + 0.006*"previou" + 0.004*"upgrad"')
(4, '0.024*"light" + 0.015*"turn" + 0.010*"echo" + 0.010*"charg"')
(5, '0.013*"charg" + 0.009*"week" + 0.006*"junk" + 0.006*"plug"')
(6, '0.063*"love" + 0.031*"tablet" + 0.031*"bought" + 0.025*"kid"')


/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


What is coherence?

A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. 

Coherence Measures

Let’s take quick look at different coherence measures, and how they are calculated:

C_v measure is based on a sliding window, one-set segmentation of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity

C_p is based on a sliding window, one-preceding segmentation of the top words and the confirmation measure of Fitelson’s coherence

C_uci measure is based on a sliding window and the pointwise mutual information (PMI) of all word pairs of the given top words

C_umass is based on document cooccurrence counts, a one-preceding segmentation and a logarithmic conditional probability as confirmation measure

C_npmi is an enhanced version of the C_uci coherence using the normalized pointwise mutual information (NPMI)

C_a is baseed on a context window, a pairwise comparison of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity

There is, of course, a lot more to the concept of topic model evaluation, and the coherence measure. However, keeping in mind the length, and purpose of this ICP, let’s apply these concepts into developing a model that is at least better than with the default parameters. 

In [30]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4233908258330707


In [31]:

#finding the similarity of the first review with topics
print('\n')
print("first review is:")
print(reviews_df.Reviews[0])
get_document_topics = ldamodel.get_document_topics(corpus[0])
print('\n')
print("The similarity of this review with the topics and respective similarity score are ")
print(get_document_topics)



first review is:
I thought it would be as big as small paper but turn out to be just like my palm. I think it is too small to read on it... not very comfortable as regular Kindle. Would definitely recommend a paperwhite instead.


The similarity of this review with the topics and respective similarity score are 
[(1, 0.2708032), (2, 0.45502993), (4, 0.25572383), (6, 0.013450081)]


In [32]:
#visualizing topics
lda_viz = gensim.models.ldamodel.LdaModel.load('/content/model_combined.gensim')
lda_display = pyLDAvis.gensim.prepare(lda_viz, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
